In [3]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error

# === 1. Cargar dataset ===
df = pd.read_csv("top_40_corr.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 🔧 Limpiar nombres de columnas
df.columns = df.columns.str.replace(r"[^\w]", "_", regex=True)

# === 2. Features y target ===
features = [col for col in df.columns if col not in ['product_id', 'periodo', 'tn']]
target = 'tn'

# === 3. Separar train y validación ===
df_train = df[df['periodo'] < '2019-12-01']
df_val = df[df['periodo'] == '2019-12-01']

X_train = df_train[features].copy()
y_train = df_train[target]
X_val = df_val[features].copy()
y_val = df_val[target]

# 🔁 4. Convertir columnas object a category ===
for col in X_train.select_dtypes(include='object').columns:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

# === 5. Función objetivo de Optuna ===
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 256),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 3.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 3.0),
        'random_state': 42
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return mean_absolute_error(y_val, y_pred)

# === 6. Ejecutar la búsqueda bayesiana con persistencia ===
study = optuna.create_study(
    direction="minimize",
    study_name="lgbm_top40",
    storage="sqlite:///optuna_lgbm_top40.db",
    load_if_exists=True
)
study.optimize(objective, n_trials=50)

# === 7. Ver resultados ===
print("✅ Mejor MAE:", study.best_value)
print("🧪 Mejores hiperparámetros:", study.best_params)


[I 2025-06-28 00:01:40,132] Using an existing study with name 'lgbm_top40' instead of creating a new one.


[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27
[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.170837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22101
[LightGBM] [Info] Number of data points in the train set: 350735, number of used features: 108
[LightGBM] [Info] Start training from score 1.244362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

[I 2025-06-28 00:01:52,672] Trial 50 finished with value: 0.05909983400611125 and parameters: {'n_estimators': 328, 'learning_rate': 0.0412148039669269, 'max_depth': 10, 'num_leaves': 234, 'subsample': 0.760086666078604, 'colsample_bytree': 0.7850247724876196, 'min_data_in_leaf': 27, 'reg_alpha': 2.0016884527431564, 'reg_lambda': 1.633470220483197}. Best is trial 43 with value: 0.04675266357114191.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=27, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=27
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.188558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22101
[LightGBM] [Info] Number of data points in the train set: 350735, number of used features: 108
[LightGBM] [Info] Start training from score 1.244362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2025-06-28 00:02:17,140] Trial 51 finished with value: 0.05298448911237518 and parameters: {'n_estimators': 483, 'learning_rate': 0.021695876517542307, 'max_depth': 10, 'num_leaves': 247, 'subsample': 0.7315401914898232, 'colsample_bytree': 0.7438308234840093, 'min_data_in_leaf': 21, 'reg_alpha': 1.7062129815741378, 'reg_lambda': 1.8866890164705385}. Best is trial 43 with value: 0.04675266357114191.


...

[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234487 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22101
[LightGBM] [Info] Number of data points in the train set: 350735, number of used features: 108
[LightGBM] [Info] Start training from score 1.244362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

[I 2025-06-28 00:25:38,284] Trial 98 finished with value: 0.05382068955425961 and parameters: {'n_estimators': 647, 'learning_rate': 0.017084849295109176, 'max_depth': 10, 'num_leaves': 206, 'subsample': 0.8238701788841458, 'colsample_bytree': 0.8309578482589509, 'min_data_in_leaf': 36, 'reg_alpha': 2.328588268306584, 'reg_lambda': 2.3065319096579953}. Best is trial 43 with value: 0.04675266357114191.


[LightGBM] [Warning] min_data_in_leaf is set=36, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=36
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.191543 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22101
[LightGBM] [Info] Number of data points in the train set: 350735, number of used features: 108
[LightGBM] [Info] Start training from score 1.244362
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2025-06-28 00:25:57,543] Trial 99 finished with value: 0.06347737299328413 and parameters: {'n_estimators': 702, 'learning_rate': 0.052950454413205374, 'max_depth': 11, 'num_leaves': 241, 'subsample': 0.704193767504527, 'colsample_bytree': 0.7363783223442574, 'min_data_in_leaf': 50, 'reg_alpha': 1.129576577600161, 'reg_lambda': 2.3628611282896546}. Best is trial 43 with value: 0.04675266357114191.


✅ Mejor MAE: 0.04675266357114191
🧪 Mejores hiperparámetros: {'n_estimators': 647, 'learning_rate': 0.010595707689344216, 'max_depth': 12, 'num_leaves': 215, 'subsample': 0.7554369080564807, 'colsample_bytree': 0.8144048873939789, 'min_data_in_leaf': 38, 'reg_alpha': 1.5224038755383693, 'reg_lambda': 2.3716401583915445}
